<a href="https://colab.research.google.com/github/ryujiuno0323/Mask_R_CNN/blob/master/Mask_R_CNN_Colab_HL60_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Mask-R-CNN_Colab_HL60_dataset

データセット　HL60 CELL LINE (FIXED CELLS)
https://horomary.hatenablog.com/entry/2019/07/28/124000


1.Google Colaboratory上で新しい実行空間を作成し、GPUを有効にする

2.GitHubからMask R-CNNのソースコードを取得し、セットアップする
GitHubからCOCO APIのソースコードを取得し、セットアップする

3.COCOデータセットで訓練済のMask R-CNN学習済モデルをダウンロードする

4.それを使ってサンプル画像を推論させて、結果を画面に表示する

In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [2]:
import tensorflow
print(tensorflow.__version__)

1.15.2


In [3]:
!pip3 install keras==2.2.5

In [ ]:
import keras
print(keras.__version__)

In [15]:
import os

p_temp=os.getcwd()
print(p_temp)
#取得したディレクトリの配下のディレクトリ確認
print(os.listdir(path=p_temp))

/content/coco/PythonAPI
['pycocotools', 'setup.py', 'pycocoDemo.ipynb', 'pycocotools.egg-info', 'mask_rcnn_coco.h5', 'build', 'logs', 'dist', 'pycocoEvalDemo.ipynb', 'Makefile']


In [ ]:
# gitからソースを取得
# 依存パッケージのインストール
#matterport/MASK_RCNNリポジトリを
#clone(https://github.com/matterport/Mask_RCNN)
%cd /content
!git clone https://github.com/matterport/Mask_RCNN.git

In [ ]:
import os

p_temp=os.getcwd()
print(p_temp)
#取得したディレクトリの配下のディレクトリ確認
print(os.listdir(path=p_temp))

#1.Mask R-CNNのソースコードを取得し、セットアップする

GitHubからMask R-CNNのソースコードを取得します。Colaboratoryが提供してくれる仮想マシン上の "/content" ディレクトリにソースがクローンされるようにします。次に、Mask R-CNNが必要とするnumpy, tensorflow, kerasなどのライブラリはリポジトリに含まれる "requirements.txt" に記載されていますので、こちらを全てpipでインストールします。最後に、リポジトリに含まれるセットアップスクリプト "setup.py" を実行して完了です。

In [4]:
# gitからソースを取得
# 依存パッケージのインストール
#matterport/MASK_RCNNリポジトリを
#clone(https://github.com/matterport/Mask_RCNN)
%cd /content
!git clone https://github.com/matterport/Mask_RCNN.git

/content
Cloning into 'Mask_RCNN'...
remote: Enumerating objects: 956, done.
remote: Total 956 (delta 0), reused 0 (delta 0), pack-reused 956
Receiving objects: 100% (956/956), 116.76 MiB | 27.82 MiB/s, done.
Resolving deltas: 100% (566/566), done.


In [ ]:
# ライブラリを取得
%cd /content/Mask_RCNN
!pip install -r requirements.txt

In [ ]:
# setup.pyを実行
#matterport/MASK_RCNNのインストール
#MASK_RCNNの最上位ディレクトリでpython setup.py install
%cd /content/Mask_RCNN
%run -i setup.py install

#2.COCO APIのソースコードを取得し、セットアップする

GitHubからCOCO APIのソースコードを取得します。リポジトリに含まれるセットアップスクリプト "setup.py" を実行して完了です。

In [7]:
# COCO 用ソースを取得
# pycocotoolsのインストール
# cocoリポジトリをclone(https://github.com/waleedka/coco)
%cd /content
!git clone https://github.com/waleedka/coco.git

/content
Cloning into 'coco'...
remote: Enumerating objects: 904, done.
remote: Total 904 (delta 0), reused 0 (delta 0), pack-reused 904
Receiving objects: 100% (904/904), 10.39 MiB | 11.55 MiB/s, done.
Resolving deltas: 100% (539/539), done.


In [ ]:
# python用APIをインストール
# matterport/MASK_RCNNのインストール
# MASK_RCNNの最上位ディレクトリでpython setup.py install

%cd /content/coco/PythonAPI
%run -i setup.py build_ext --inplace
%run -i setup.py build_ext install

Configクラスについて

OneClassConfigではmrcnn.configからConfigクラスを継承して訓練の設定を行います。
※バッチ当たりの画像数は増やしすぎるとGPUメモリに乗りません。

In [9]:
from mrcnn.config import Config


class OneClassConfig(Config):

    #: config名
    NAME = "cell_dataset"

    #: バッチあたり画像数 (GPUのメモリが大きいなら増やしてもよい
    IMAGES_PER_GPU = 1

    # クラス数　= 背景 + 検出クラス数
    NUM_CLASSES = 1 + 1

    # エポックあたりステップ数
    STEPS_PER_EPOCH = 50

    VALIDATION_STEPS = 5

    # 提案領域のconfidenceが90%以下なら物体検出フェイズをスキップ
    DETECTION_MIN_CONFIDENCE = 0.9

Datasetクラスについて

OneClassDatasetではmrcnn.utilsからDatasetクラスを継承して画像読み込みメソッドの定義ととmask生成のメソッドのオーバーライドを行います。

In [10]:
import pathlib

import cv2
from PIL import Image
from mrcnn import utils
from mrcnn.model import log


class OneClassDataset(utils.Dataset):

    def load_dataset(self, dataset_dir):
        """ データセットを登録
        """
        #: データセット名、クラスID、クラス名
        self.add_class('cell_dataset', 1, 'cell')

        images = glob.glob(os.path.join(dataset_dir, "image", "*.jpg"))
        masks = glob.glob(os.path.join(dataset_dir, "mask", "*.jpg"))

        for image_path, mask_path in zip(images, masks):
            image_path = pathlib.Path(image_path)
            mask_path = pathlib.Path(mask_path)
            assert image_path.name == mask_path.name, 'データセット名不一致'

            image = Image.open(image_path)
            height = image.size[0]
            width = image.size[1]

            mask = Image.open(mask_path)
            assert image.size == mask.size, "サイズ不一致"

            self.add_image(
                'cell_dataset',
                path=image_path,
                image_id=image_path.stem,
                mask_path=mask_path,
                width=width, height=height)

    def load_mask(self, image_id):
        """マスクデータとクラスidを生成する
        """
        image_info = self.image_info[image_id]
        if image_info["source"] != 'cell_dataset':
            return super(self.__class__, self).load_mask(image_id)

        mask_path = image_info['mask_path']
        mask, cls_idxs = blob_detection(str(mask_path))

        return mask, cls_idxs

    def image_reference(self, image_id):
        """Return the path of the image."""
        info = self.image_info[image_id]
        if info["source"] == 'cell_dataset':
            return info
        else:
            super(self.__class__, self).image_reference(image_id)


def blob_detection(mask_path):
    mask = cv2.imread(mask_path, 0)
   #: 念のためもう一度二値化
    _, mask = cv2.threshold(mask, 150, 255, cv2.THRESH_BINARY)

    label = cv2.connectedComponentsWithStats(mask)
    data = copy.deepcopy(label[1])

    labels = []
    for label in np.unique(data):
        #: ラベル0は背景
        if label == 0:
            continue
        else:
            labels.append(label)

    mask = np.zeros((mask.shape)+(len(labels),), dtype=np.uint8)

    for n, label in enumerate(labels):
        mask[:, :, n] = np.uint8(data == label)

    cls_idxs = np.ones([mask.shape[-1]], dtype=np.int32)

    return mask, cls_idxs

Using TensorFlow backend.


In [11]:
import os

p_temp=os.getcwd()
print(p_temp)
#取得したディレクトリの配下のディレクトリ確認
print(os.listdir(path=p_temp))
#絶対パス
#file = os.path.abspath("sample_data")
#file = os.path.abspath("model.py")
file = os.path.abspath("mrcnn/model.py")


print(file)

/content/coco/PythonAPI
['pycocotools', 'setup.py', 'pycocoDemo.ipynb', 'pycocotools.egg-info', 'build', 'dist', 'pycocoEvalDemo.ipynb', 'Makefile']
/content/coco/PythonAPI/mrcnn/model.py


学習実行

モデル訓練のコードです。cocoデータセット学習済み重みをモデルへロードした後、DatasetクラスとConfigクラスをモデルへ渡して訓練開始。
※DatasetクラスとConfigクラスの詳細は後述

In [13]:
    import os
    import glob

    import mrcnn.model as modellib
    from mrcnn import utils


    TRAIN_DATASET = os.path.join('dataset', 'train')
    dataset_train = OneClassDataset()
    dataset_train.load_dataset(TRAIN_DATASET)
    dataset_train.prepare()

    VALID_DATASET = os.path.join('datset', 'valid')
    dataset_val = OneClassDataset()
    dataset_val.load_dataset(VALID_DATASET)
    dataset_val.prepare()

    config = OneClassConfig()


    model = modellib.MaskRCNN(mode="training", config=config,
                              model_dir="logs/model")

    COCO_MODEL_PATH = 'mask_rcnn_coco.h5'
    if not os.path.exists(COCO_MODEL_PATH):
        utils.download_trained_weights(COCO_MODEL_PATH)
    
    model.load_weights(COCO_MODEL_PATH, by_name=True,
                       exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",
                                "mrcnn_bbox", "mrcnn_mask"])

   #: ネットワークのhead部分のみの訓練
    model.train(dataset_train, dataset_val,
                learning_rate=0.001,
                epochs=10,
                layers='heads')


Starting at epoch 0. LR=0.001

Checkpoint Path: logs/model/cell_dataset20200619T1008/mask_rcnn_cell_dataset_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_class_bn2        (TimeDistributed)
mrcnn_mask_conv4       (TimeDistributed)
mrcnn_mask_bn4

/tensorflow-1.15.2/python3.6/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/tensorflow-1.15.2/python3.6/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/tensorflow-1.15.2/python3.6/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.6/dist-packages/keras/engine/training_generator.py:49: UserWarning: Using a generator with `use_multiprocessing=True` and multiple 

Epoch 1/10


UnboundLocalError: ignored

結果の確認

validation データセットについて結果の確認を行います。

In [14]:
    import random
    from mrcnn import visualize

    VALID_DATASET = os.path.join('dataset', 'valid')
    dataset_val = OneClassDataset()
    dataset_val.load_dataset(VALID_DATASET)
    dataset_val.prepare()

    config = InferenceConfig()
    MODEL_DIR = os.path.join("logs", "model")
    model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR,
                              config=config)
    
    weights_path = model.find_last()
    print("Loading weights ", weights_path)
    model.load_weights(weights_path, by_name=True)

    
    random.choice(dataset_val.image_ids)
    image, image_meta, gt_class_id, gt_bbox, gt_mask = modellib.load_image_gt(dataset_val, config, image_id)
    info = dataset_val.image_info[image_id]
    
    results = model.detect([image], verbose=1)

    # Display results
    ax = get_ax(1)
    r = results[0]
    visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'],
                                dataset_val.class_names, r['scores'], ax=ax,
                                title="Predictions")

NameError: ignored

#3.COCOデータセットで訓練済のMask R-CNN学習済モデルをダウンロードする

ここからはPythonを使ったコーディングになります。同様のソースコードが、Mask R-CNNリポジトリの "samples/demo.ipynb" にあるため、そちらをコピペして実行しても同じ結果が得られます。その場合、冒頭のROOT_DIR変数を仮想マシン上のパスである "/content/Mask_RCNN" に書き換える必要があるため、注意しましょう。

以下は、各種初期設定と、COCOデータセットで訓練済のMask R-CNN学習済モデルをダウンロードするまでの一連の処理となります。

In [ ]:
import os
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt

# Root directory of the project
ROOT_DIR = os.path.abspath("/content/Mask_RCNN")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
# Import COCO config
sys.path.append(os.path.join(ROOT_DIR, "samples/coco/"))  # To find local version
import coco

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

# Directory of images to run detection on
IMAGE_DIR = os.path.join(ROOT_DIR, "images")

SyntaxError: ignored

#4.サンプル画像を推論して、結果を画面に表示する

これで全ての準備が整ったため、Mask R-CNNリポジトリに含まれるサンプル画像を読み込ませて推論させていきます。

In [ ]:
class InferenceConfig(coco.CocoConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
# config.display()


# Create model object in inference mode.
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

# Load weights trained on MS-COCO
model.load_weights(COCO_MODEL_PATH, by_name=True)


# COCO Class names
# Index of the class in the list is its ID. For example, to get ID of
# the teddy bear class, use: class_names.index('teddy bear')
class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
               'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
               'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
               'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
               'kite', 'baseball bat', 'baseball glove', 'skateboard',
               'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
               'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
               'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
               'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
               'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
               'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
               'teddy bear', 'hair drier', 'toothbrush']


# Load a random image from the images folder
file_names = next(os.walk(IMAGE_DIR))[2]
for file_name in file_names:
  image = skimage.io.imread(os.path.join(IMAGE_DIR, file_name))

  # Run detection
  results = model.detect([image], verbose=1)

  # Visualize results
  r = results[0]
  visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                              class_names, r['scores'])